In [1]:
import csv
import ArgConfigParse
from dateutil import rrule
from datetime import datetime
import logging
from pathlib import Path

from sys import exit

In [2]:
def do_exit(msg=None, level=0):
    if level > 0:
        
        print(f'exiting due to error:\n     {msg}')
    elif msg:
        print(f'{msg}')
        print('exiting')
    exit(level)
    

In [3]:
def parse_schedules(schedule_dict):
    # parse schedule file into lookup tables
    if not isinstance(schedule_dict, dict):
        raise TypeError (f'schedule_dict must be of type(dict), not {type(schedule_dict)}')
    schedule_mttf = {} 
    schedule_w = {}
    for key, value in schedule_dict:
        if key.startswith('%'):
            if 'wednesday' in key:
                schedule_w[key] = value
            else:
                schedule_mttf[key] = value
        else:
            logging.warning(f'unrecongized "day" item in confiuration file(s) {schedule.config_files}: {key}')
            logging.info(f'all "day" items must follow the format "[%day N] or "[%day N wednesday]"') 
    return(schedule_mttf, schedule_w)

In [4]:
def read_vacations(vacations):
    '''return list of daytime objects based on dates in `vacations` file'''
    # read vacation/non instructional days list
    with open(vacations, 'r') as vacation_file:
        vacation_txt = vacation_file.readlines()

    vacation_dt = []
    for i in vacation_txt:
        try:
            vacation_dt.append(datetime.strptime(i.strip(), '%Y/%m/%d'))
        except ValueError as e:
            logging.debug(f'skipping unknown date format in {vacations}: "{i}""')
        
    return vacation_dt

    

In [5]:
def set_school_days(start, end, vacation, dt_format):
    
    start_dt = datetime.strptime(start, dt_format)
    end_dt = datetime.strptime(end, dt_format)
    
    # set actual school days
    school_days = []
    for dt in rrule.rrule(rrule.DAILY, dtstart=start_dt, until=end_dt):
        if dt not in vacation and datetime.weekday(dt) in range(0, 5):
                school_days.append(dt)
                
    return school_days
        

In [62]:
def main():
    logging.root.setLevel('INFO')
    config = ArgConfigParse.ConfigFile(['setup.ini'])
    schedule = ArgConfigParse.ConfigFile(['schedule.ini'])
    vacation_file = Path('./vacation_days.txt').absolute().expanduser()

    # parse config files
    config.parse_config()
    schedule.parse_config()
    main_cfg = config.config_dict
    sy_start = main_cfg['dates']['start']
    sy_end = main_cfg['dates']['end']
    dt_format = main_cfg['main']['dt_format']
    
        
    # build lookup tables for mttf, w schedules
    try:
        schedule_mttf, schedule_w = parse_schedules(schedule.config_dict)
    except Exception as e:
        do_exit(msg=f'failed to read schedule file: {schedule.config_files}')

    vacation = read_vacations(vacation_file)
    school_days = set_school_days(start=sy_start, 
                                  end=sy_end, vacation=vacation, 
                                  dt_format=dt_format)
    
    return (school_days, schedule_mttf, schedule_w)

In [63]:
if __name__ == '__main__':
    s = main()

In [64]:
days, s_mttf, s_w = s

In [75]:
for i, val in enumerate(days):
    if val.weekday() == 2:
        my_schedule = s_w
    else:
        my_schedule = s_mttf
#     print(f'schl_day: {i+1}, rot_day: {i%8+1}, schedule: {sorted(my_schedule)[i%8]}')
    print(f'{datetime.strftime(val, "%Y/%m/%d")}, {sorted(my_schedule)[i%8]}')

2021/08/18, %day 1 wednesday
2021/08/19, %day 2
2021/08/20, %day 3
2021/08/23, %day 4
2021/08/24, %day 5
2021/08/25, %day 6 wednesday
2021/08/26, %day 7
2021/08/27, %day 8
2021/08/30, %day 1
2021/08/31, %day 2
2021/09/01, %day 3 wednesday
2021/09/02, %day 4
2021/09/03, %day 5
2021/09/06, %day 6
2021/09/07, %day 7
2021/09/08, %day 8 wednesday
2021/09/09, %day 1
2021/09/10, %day 2
2021/09/13, %day 3
2021/09/14, %day 4
2021/09/15, %day 5 wednesday
2021/09/16, %day 6
2021/09/17, %day 7
2021/09/20, %day 8
2021/09/21, %day 1
2021/09/22, %day 2 wednesday
2021/09/23, %day 3
2021/09/27, %day 4
2021/09/28, %day 5
2021/09/29, %day 6 wednesday
2021/09/30, %day 7
2021/10/01, %day 8
2021/10/04, %day 1
2021/10/05, %day 2
2021/10/06, %day 3 wednesday
2021/10/07, %day 4
2021/10/08, %day 5
2021/10/11, %day 6
2021/10/12, %day 7
2021/10/13, %day 8 wednesday
2021/10/14, %day 1
2021/10/15, %day 2
2021/10/25, %day 3
2021/10/26, %day 4
2021/10/27, %day 5 wednesday
2021/10/28, %day 6
2021/10/29, %day 7
2021/11